# Prerequisites

In [1]:
# http://zetcode.com/db/sqlitepythontutorial/

import sqlite3 as lite
import sys

# Version

In [2]:
with lite.connect('test.db') as con:
    cur = con.cursor()    
    cur.execute('SELECT SQLITE_VERSION()')
    
    data = cur.fetchone()
    
    print("SQLite version: %s" % data)

SQLite version: 3.14.2


# Inserting data

In [3]:
cars = ((1, 'Audi', 52642),
        (2, 'Mercedes', 57127),
        (3, 'Skoda', 9000),
        (4, 'Volvo', 29000),
        (5, 'Bentley', 350000),
        (6, 'Hummer', 41400),
        (7, 'Volkswagen', 21600))

with lite.connect('test.db') as con:
    cur = con.cursor()
    cur.execute("DROP TABLE IF EXISTS Cars")
    cur.execute("CREATE TABLE Cars(Id INT, Name TEXT, Price INT)")
    cur.executemany("INSERT INTO Cars VALUES(?, ?, ?)", cars)   

# The last inserted row id

In [4]:
with lite.connect(':memory:') as con:
    cur = con.cursor()    
    cur.execute("CREATE TABLE Friends(Id INTEGER PRIMARY KEY, Name TEXT);")
    cur.execute("INSERT INTO Friends(Name) VALUES ('Tom');")
    cur.execute("INSERT INTO Friends(Name) VALUES ('Rebecca');")
    cur.execute("INSERT INTO Friends(Name) VALUES ('Jim');")
    cur.execute("INSERT INTO Friends(Name) VALUES ('Robert');")
    
    lid = cur.lastrowid
    print("The last Id of the inserted row is %d" % lid)

The last Id of the inserted row is 4


# Retrieving data

In [5]:
with lite.connect('test.db') as con:
    cur = con.cursor()
    cur.execute("SELECT * FROM Cars")
    for row in cur.fetchall():
        print(row)

(1, 'Audi', 52642)
(2, 'Mercedes', 57127)
(3, 'Skoda', 9000)
(4, 'Volvo', 29000)
(5, 'Bentley', 350000)
(6, 'Hummer', 41400)
(7, 'Volkswagen', 21600)


# The dictionary cursor

In [6]:
with lite.connect('test.db') as con:
    con.row_factory = lite.Row
    
    cur = con.cursor()
    cur.execute("SELECT * FROM Cars")
    
    for row in cur.fetchall():
        print("%s %s %s" % (row["Id"], row["Name"], row["Price"]))

1 Audi 52642
2 Mercedes 57127
3 Skoda 9000
4 Volvo 29000
5 Bentley 350000
6 Hummer 41400
7 Volkswagen 21600


# Parameterized queries

In [7]:
with lite.connect('test.db') as con:
    cur = con.cursor()
    cur.execute("UPDATE Cars SET Price=? WHERE Id=?", (62300, 1))
    con.commit()
    
    print("Number of rows updated: %d" % cur.rowcount)
    
    cur.execute("SELECT Name, Price FROM Cars WHERE Id=:Id", {"Id": 1})
    con.commit()
    
    row = cur.fetchone()
    print(row[0], row[1])


Number of rows updated: 1
Audi 62300


# Metadata

In [8]:
with lite.connect('test.db') as con:
    cur = con.cursor()
    
    # The PRAGMA table_info(tableName) command returns one row 
    # for each column in the Cars table. 
    
    cur.execute('PRAGMA table_info(Cars)')
    data = cur.fetchall()
    for d in data:
        print(d[0], d[1], d[2])
        
    # We list all tables in the test.db database.
    
    cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
    rows = cur.fetchall()
    print()
    for row in rows:
        print(row[0])

0 Id INT
1 Name TEXT
2 Price INT

Cars


# Export and import of data

## Export data

In [11]:
cars = ((1, 'Audi', 52643),
        (2, 'Mercedes', 57642),
        (3, 'Skoda', 9000),
        (4, 'Volvo', 29000),
        (5, 'Bentley', 350000),
        (6, 'Hummer', 41400),
        (7, 'Volkswagen', 21600))

with lite.connect(':memory:') as con:
    cur = con.cursor()
    
    cur.execute("DROP TABLE IF EXISTS Cars")
    cur.execute("CREATE TABLE Cars(Id INT, Name TEXT, Price INT)")
    cur.executemany("INSERT INTO Cars VALUES(?, ?, ?)", cars)
    cur.execute("DELETE FROM Cars WHERE Price < 30000")
    
    data = '\n'.join(con.iterdump())
    
    with open('car.sql', 'w') as f:
        f.write(data)
    
    

## Import data

In [13]:
# Dumped table back into memory.

with lite.connect(':memory:') as con:
    cur = con.cursor()
    
    with open('car.sql', 'r') as f:
        sql = f.read()
        cur.executescript(sql)
        
    cur.execute("SELECT * FROM Cars")
    rows = cur.fetchall()
    for row in rows:
        print(row)
    
    

(1, 'Audi', 52643)
(2, 'Mercedes', 57642)
(5, 'Bentley', 350000)
(6, 'Hummer', 41400)
